In [4]:
import pandas as pd 
import numpy as np

import xml.etree.ElementTree as ET

In [5]:
spacelist=['','','']

plan=[
['P1',200,200,250,250],
['P2',150,150,150,150],
['P3',100,100,100,100]]
plan=pd.DataFrame(plan,columns=['Pr',1,  2,  3,  4  ])
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)

periode=5 #--------------------------------------------------

In [6]:
tree = ET.parse(f'XMLs\\periode_{periode-1}.xml')
root = tree.getroot()

waitinglist=list(np.zeros(15)) 

for a in root.find('waitinglistworkstations').findall('workplace'):
    waitinglist[int(a.attrib['id'])-1]+=int(a.attrib['timeneed'])

for a in root.find('ordersinwork').findall('workplace'):#.findall('workplace'):
    #print(a.attrib['timeneed'])
    waitinglist[int(a.attrib['id'])-1]=waitinglist[int(a.attrib['id'])-1]+int(a.attrib['timeneed'])


timeneeddf = pd.DataFrame(spacelist.copy()+waitinglist).T

In [7]:
arbeitsplaetze=15
index_l=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,26,49,54,29,50,55,30,51,56,31,1,2,3]
u=['K','D','H','K','D','H','K','D','H','K','D','H','KDH','KDH','K','D','H','KDH','K','D','H','K','D','H','K','D','H','K','D','H']

combis=[ 
    [ [6,18,19,20] ],
    [ [5,21,22,23] ],
    [ [5,24],[6,25,26] ],
    [ [6,27],[7,28,29] ],
    [ [0] ],
    [ [3,16,15,14], [2,12]],
    [ [2,16,15,14,17,11,10,9,8,7,6] ],
    [ [1,6,9], [2, 11,10,8,7], [3,16,15,14 ] ],
    [ [3,11,10,8,7,6,9],[2,16,15,14 ] ],
    [ [4,0,1,2,3,4,5]],
    [ [3,0,1,2,3,4,5]],
    [ [3,11,10,8,7,6,9]],
    [ [2,11,10,8,7,6,9]],
    [ [3,12]],
    [ [3,13,17]],
]
def f(x):
    plan_dict={
        'K': plan[1][0],
        'D': plan[1][1],
        'H': plan[1][2],
        'KDH': plan[1][0]+plan[1][1]+plan[1][2]
    }
    return plan_dict[x]

main_df=pd.DataFrame(np.array([u,index_l]).T,columns=['u','NR'])
#main_df['menge']=main_df['menge'].astype(float)
main_df['menge']=main_df['u'].apply(f)


df=pd.DataFrame()
for x in range(1,arbeitsplaetze+1):

    col=np.zeros(len(index_l))

    if x<len(combis)+1:
        for a in combis[x-1]:
            val=a.pop(0)
            
            for y in a:
                col[y]=val

    df[x]=col

for x in range(1,arbeitsplaetze+1):
    main_df[x]=df[x]*main_df['menge']

main_df.index=main_df['NR']


In [8]:
calc_l=spacelist.copy()
for x in range(1,arbeitsplaetze+1):
    calc_l.append(sum(main_df[x]))
#-------------------------   1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
rüstzeit=spacelist.copy()+  [60,60,60,60,0,60,400,360,360,120,120,120,120,20,40]
rüstzeit_p=spacelist.copy()+[0,0,0,0,0,0,50,0,50,0,0,0,0,0,50]

timeneeddf.columns=main_df.columns
bottom=pd.DataFrame([calc_l,rüstzeit,timeneeddf.to_numpy()[0],rüstzeit_p],columns=main_df.columns)

bottom=bottom.append(bottom.sum().to_frame().T)
bottom.index=['timeneed','rüstzeit','timeneed_p-1','rüstzeit_p-1','total_need']
bottom=bottom.T    

In [9]:
def ueberstunden(x):
    if type(x)==str:
        return '','',''  
    #Eine Schicht ohne Ü
    if x <= 2400:
        return 0,0,1
    #Überstunden 1 schicht
    if (x > 2400) & ( x <= 3600 ):#--------------------------------------------------------------------------------
        return x-2400,(x-2400)/5,1
    #Zwei schichten ohne Ü
    if (x > 2400) & (x <= 4800):
        return 0,0,2
    #Zwei Schichten Überstunden 
    if (x > 4800) & (x <= 6000):
        return x-4800,(x-4800)/5,2
    if (x > 6000) & (x <= 7200):
        return 0,0,3
    if x > 7200:
        return 99999,99999,3
    
    
bottom['Überstunden_total'],bottom['überstunden_tag'],bottom['Schichten']=zip( *bottom['total_need'].apply(ueberstunden) )

production_df=pd.concat([main_df,bottom.T])
production_df.to_csv(f'Production\\prod_{periode}.csv')
production_df

,u,NR,menge,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
4,K,4,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800.0,600.0,0.0,0.0,0.0,0.0
5,D,5,150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,450.0,0.0,0.0,0.0,0.0
6,H,6,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
7,K,7,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800.0,600.0,0.0,0.0,0.0,0.0
8,D,8,150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,450.0,0.0,0.0,0.0,0.0
9,H,9,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
10,K,10,200,0.0,0.0,0.0,0.0,0.0,0.0,400.0,200.0,600.0,0.0,0.0,600.0,400.0,0.0,0.0
11,D,11,150,0.0,0.0,0.0,0.0,0.0,0.0,300.0,300.0,450.0,0.0,0.0,450.0,300.0,0.0,0.0
12,H,12,100,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,300.0,0.0,0.0,300.0,200.0,0.0,0.0
13,K,13,200,0.0,0.0,0.0,0.0,0.0,0.0,400.0,200.0,600.0,0.0,0.0,600.0,400.0,0.0,0.0
